# CHEAT SHEET - LANGGRAPH SESIÓN 1

**Módulo 8: Introducción a LangGraph**

---
## CONCEPTOS CLAVE

### LangChain vs LangGraph - Comparativa Rápida

| Aspecto | LangChain | LangGraph |
|---------|-----------|----------|
| **Metáfora** | 🚂 Tren en vías fijas | 🗺️ Red de carreteras con GPS inteligente |
| **Estructura** | Cadenas lineales (A→B→C) | Grafos con ciclos y bifurcaciones |
| **Estado** | Temporal, pasa entre pasos | **Memoria de corto plazo** (post-it en reunión) |
| **Condicionales** | Limitado (RouterChain) | Nativo con edges condicionales |
| **Ciclos** | ❌ No soportados | ✅ Diseñados para razonamiento iterativo |
| **Multi-agente** | Complejo de implementar | Built-in |
| **Recuperación** | Manual | Automática con persistencia |
| **Caso ideal** | RAG simple, ETL, traducciones | Agentes con retry logic, decisiones dinámicas, human-in-the-loop |

**Regla de oro:** Si necesitas **volver atrás, tomar decisiones en tiempo real o colaborar con múltiples agentes** → **LangGraph**

---
## ANATOMÍA DE UN GRAFO

```
┌─────────────────────────────────────────────────┐
│             LANGGRAPH = ARQUITECTURA            │
│                                                 │
│  ┌──────────┐         ┌──────────┐              │
│  │  NODO A  │────────>│  NODO B  │              |
│  │(Capacidad│         │(Capacidad│              │
│  └──────────┘         └────┬─────┘              │
│       ▲                    │                    │
│       │                    ▼                    │
│       │              ┌──────────┐               │
│       │              │  ROUTER  │               │
│       │              │(Decisión)│               │
│       │              └────┬─────┘               │
│       │                   │                     │
│       │        ┌──────────┴──────────┐          │
│       │        │                     │          │
│       │        ▼                     ▼          │
│       │   ┌────────┐            ┌────────┐      │
│       └───│ NODO C │            │ NODO D │      │
│           │ (Loop) │            │ (Fin)  │      │
│           └────────┘            └────────┘      │
│                                                 │
│  ESTADO (post-it compartido):                   │
│  { "dato": "valor", "intentos": 2 }             │
└─────────────────────────────────────────────────┘
```

### Componentes Fundamentales

**1. ESTADO (State) - La Memoria de Corto Plazo**
- Diccionario compartido (como un **post-it en reunión**)
- Todos los nodos pueden leerlo y escribir en él
- Persiste durante la ejecución del grafo
- NO es una base de datos permanente (para eso usa checkpointing o DB externas)

**2. NODO (Node) - Unidad de Capacidad**
- Función que representa **una responsabilidad** específica
- Regla: "1 nodo = 1 capacidad" (si hace dos cosas, son dos nodos)
- Recibe estado → Ejecuta lógica → Devuelve actualizaciones

**3. EDGE (Arista/Conexión) - Flujo de Información**
- **Incondicional:** Siempre va de A a B (`add_edge`)
- **Condicional:** Usa un router para decidir destino (`add_conditional_edges`)

**4. ROUTER - GPS Inteligente**
- Función que decide el próximo nodo según el estado
- Devuelve un **string** con el nombre del nodo destino

---
## RECETARIO DE CÓDIGO

### 1. Definir un Estado (TypedDict)

In [ ]:
from typing import TypedDict, List

class EstadoAgente(TypedDict):
    mensajes: List[str]      # Historial
    pregunta: str            # Input usuario
    respuesta: str           # Output generado
    intentos: int            # Contador

**Reglas importantes:**
- ✅ **1 nodo = 1 responsabilidad** (regla de oro de Mastra)
- ✅ Devuelve solo los campos que cambias
- ✅ Tipo de entrada = Tipo de salida

### 2. Plantilla de Función Nodo

In [ ]:
def nodo_ejemplo(state: EstadoAgente) -> EstadoAgente:
    """
    Descripción de qué hace este nodo.
    """
    # 1. Leer datos del estado
    dato = state["pregunta"]
    
    # 2. Realizar lógica (LLM, API, procesamiento)
    resultado = procesar(dato)
    
    # 3. Devolver SOLO las actualizaciones (no todo el estado)
    return {"respuesta": resultado}

**Reglas importantes:**
- ✅ Devuelve solo los campos que cambias
- ✅ Tipo de entrada = Tipo de salida
- ✅ Una responsabilidad por nodo

### 3. Plantilla de Función Router

In [ ]:
def router_ejemplo(state: EstadoAgente) -> str:
    """
    Decide a qué nodo ir según condiciones.
    DEBE devolver un STRING con el nombre del nodo.
    """
    if state["intentos"] >= 3:
        return "nodo_finalizar"  # Máximo de intentos
    elif state["respuesta"]:
        return "nodo_validar"    # Hay respuesta, validar
    else:
        return "nodo_generar"    # No hay respuesta, generar

**Reglas importantes:**
- ✅ Devuelve string, NO el nodo
- ✅ El string debe existir en el `path_map`
- ✅ Cubre todos los casos posibles

### 4. Compilar e Invocar un Grafo

In [ ]:
from langgraph.graph import StateGraph, END

# PASO 1: Crear instancia
workflow = StateGraph(EstadoAgente)

# PASO 2: Añadir nodos
workflow.add_node("procesar", nodo_procesar)
workflow.add_node("validar", nodo_validar)
workflow.add_node("finalizar", nodo_finalizar)

# PASO 3: Definir entry point
workflow.set_entry_point("procesar")

# PASO 4: Conectar edges
# Edge incondicional
workflow.add_edge("procesar", "validar")

# Edge condicional
workflow.add_conditional_edges(
    "validar",
    router_validacion,
    {
        "exito": "finalizar",
        "reintentar": "procesar"  # CICLO
    }
)

# PASO 5: Conectar finales a END
workflow.add_edge("finalizar", END)

# PASO 6: Compilar
app = workflow.compile()

# PASO 7: Ejecutar
estado_inicial = {"pregunta": "¿Hola?", "respuesta": "", "intentos": 0}
resultado = app.invoke(estado_inicial)
print(resultado)

---
## GLOSARIO RÁPIDO

| Término | Definición |
|---------|------------|
| **State** | Memoria de corto plazo del agente (post-it compartido entre nodos) que persiste durante la ejecución |
| **Node** | Unidad de capacidad: función que recibe estado, ejecuta UNA responsabilidad y devuelve actualizaciones |
| **Edge** | Conexión entre nodos que define el flujo de información |
| **Conditional Edge** | Conexión que usa un router para decidir dinámicamente el próximo nodo |
| **Router** | GPS inteligente: función que evalúa el estado y devuelve el nombre del próximo nodo como string |
| **Entry Point** | Primer nodo que se ejecuta al invocar el grafo |
| **END** | Constante que marca el final del grafo (imported from langgraph.graph) |
| **Ciclo** | Edge que apunta a un nodo anterior, permitiendo razonamiento iterativo |
| **StateGraph** | Clase principal para construir grafos en LangGraph |
| **invoke()** | Método para ejecutar un grafo compilado con un estado inicial |
| **compile()** | Método que convierte el workflow definido en un grafo ejecutable |

In [ ]:
workflow = StateGraph(Estado)
workflow.add_node("A", nodo_a)
workflow.add_node("B", nodo_b)
workflow.add_node("C", nodo_c)

workflow.set_entry_point("A")
workflow.add_edge("A", "B")
workflow.add_edge("B", "C")
workflow.add_edge("C", END)

app = workflow.compile()

**Casos de uso:**
- Validar → Procesar → Guardar
- Leer → Transformar → Escribir
- Extraer → Limpiar → Analizar

### Patrón 2: Ciclo Básico (Retry Logic)

**Estructura:**
```
         ┌──────────────┐
         ▼              │
[START] → [Procesar] ───┘ (si falla)
              │
              ▼ (si éxito)
           [END]
```

In [ ]:
def router_retry(state: Estado) -> str:
    if state["exitoso"]:
        return "finalizar"
    elif state["intentos"] >= 3:
        return "finalizar"
    else:
        return "reintentar"  # Vuelve a procesar

workflow = StateGraph(Estado)
workflow.add_node("procesar", nodo_procesar)
workflow.add_node("finalizar", nodo_finalizar)

workflow.set_entry_point("procesar")
workflow.add_conditional_edges(
    "procesar",
    router_retry,
    {
        "reintentar": "procesar",  # ¡CICLO!
        "finalizar": "finalizar"
    }
)
workflow.add_edge("finalizar", END)

app = workflow.compile()

**Casos de uso:**
- Generar contraseña hasta que sea válida
- Intentar autenticación hasta agotar intentos
- Buscar datos hasta encontrar suficientes

---
## 💡 TIP FINAL - Metodología de Diseño

**Antes de escribir código**, responde estas preguntas (del libro Patterns de Mastra):

1. **¿Qué capacidades necesita mi agente?** → Lista todas las cosas que debe saber hacer
2. **¿Cómo agrupo esas capacidades?** → Capacidades similares = mismo nodo
3. **¿Qué información necesito compartir?** → **Estado** (memoria de corto plazo)
4. **¿Qué decisiones tomo en el camino?** → **Routers**
5. **¿Necesito reintentar algo?** → **Ciclos**

**Regla de oro:** Diseña tu grafo como si estuvieras contratando un equipo humano. Cada nodo es un empleado con una responsabilidad clara.

**¡Buena suerte construyendo arquitecturas cognitivas!** 🎉

In [ ]:
def router_categoria(state: Estado) -> str:
    if state["categoria"] == "A":
        return "nodo_a"
    elif state["categoria"] == "B":
        return "nodo_b"
    else:
        return "nodo_c"

workflow = StateGraph(Estado)
workflow.add_node("clasificar", nodo_clasificar)
workflow.add_node("nodo_a", nodo_a)
workflow.add_node("nodo_b", nodo_b)
workflow.add_node("nodo_c", nodo_c)

workflow.set_entry_point("clasificar")
workflow.add_conditional_edges(
    "clasificar",
    router_categoria,
    {
        "nodo_a": "nodo_a",
        "nodo_b": "nodo_b",
        "nodo_c": "nodo_c"
    }
)

# Todos convergen a END
workflow.add_edge("nodo_a", END)
workflow.add_edge("nodo_b", END)
workflow.add_edge("nodo_c", END)

app = workflow.compile()

**Casos de uso:**
- Clasificar sentimiento → Respuesta personalizada
- Validar stock → Procesar o cancelar
- Verificar edad → Aprobar, rechazar o escalar

---
## GLOSARIO RÁPIDO

| Término | Definición |
|---------|------------|
| **State** | Diccionario compartido que persiste entre nodos y almacena toda la información del flujo |
| **Node** | Función que recibe el estado, ejecuta lógica y devuelve actualizaciones |
| **Edge** | Conexión entre nodos que define el flujo de ejecución |
| **Conditional Edge** | Conexión que usa un router para decidir dinámicamente el próximo nodo |
| **Router** | Función que evalúa el estado y devuelve el nombre del próximo nodo como string |
| **Entry Point** | Primer nodo que se ejecuta al invocar el grafo |
| **END** | Constante que marca el final del grafo (imported from langgraph.graph) |
| **Ciclo** | Edge que apunta a un nodo anterior, permitiendo iteración deliberada |
| **StateGraph** | Clase principal para construir grafos en LangGraph |
| **invoke()** | Método para ejecutar un grafo compilado con un estado inicial |
| **compile()** | Método que convierte el workflow definido en un grafo ejecutable |

---
## ⚠️ ERRORES COMUNES Y SOLUCIONES

| Error | Causa | Solución |
|-------|-------|----------|
| `KeyError: 'campo'` | Intentas leer un campo que no existe en el estado | Usa `state.get("campo", default)` o verifica que el campo se inicializa |
| `Router devuelve None` | El router no cubre todos los casos | Añade un `else: return "default"` al final del router |
| `Grafo se cuelga` | Ciclo sin condición de salida | Verifica que el router tenga una ruta a END o un contador límite |
| `Nodo no conectado` | Olvidaste añadir un edge hacia o desde el nodo | Verifica que todos los nodos tengan al menos un edge |
| `path_map inválido` | El router devuelve un string que no existe en path_map | Asegúrate de que todos los valores de retorno están en el diccionario |
| `Estado no se actualiza` | El nodo no devuelve las actualizaciones | Verifica que el nodo devuelve un dict, no None |

---
## CHECKLIST DE CONSTRUCCIÓN

Antes de ejecutar tu grafo, verifica:

- [ ] ✅ Estado definido con `TypedDict`
- [ ] ✅ Todos los nodos reciben y devuelven el tipo de estado correcto
- [ ] ✅ Entry point definido con `set_entry_point()`
- [ ] ✅ Todos los nodos añadidos con `add_node()`
- [ ] ✅ Todos los nodos tienen al menos un edge entrante (excepto entry point)
- [ ] ✅ Todos los nodos tienen al menos un edge saliente
- [ ] ✅ Nodos finales conectados a `END`
- [ ] ✅ Routers devuelven strings que existen en `path_map`
- [ ] ✅ Ciclos tienen condición de salida clara
- [ ] ✅ Grafo compilado con `.compile()`

---
## COMANDOS ÚTILES

### Visualizar Grafo (ASCII)

In [ ]:
print(app.get_graph().draw_ascii())

### Visualizar Grafo (Mermaid/PNG)

In [ ]:
from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))

### Debugging: Ver Flujo Completo

In [ ]:
# Añade prints en cada nodo
def nodo_debug(state: Estado) -> Estado:
    print(f"🔍 [NODO_DEBUG] Estado actual: {state}")
    # ... tu lógica ...
    print(f"✅ [NODO_DEBUG] Estado actualizado: {actualizaciones}")
    return actualizaciones

### Ejecutar con Estado Inicial Específico

In [ ]:
estado_inicial = {
    "campo1": "valor1",
    "campo2": 0,
    "campo3": []
}

resultado = app.invoke(estado_inicial)
print(f"Estado final: {resultado}")

---
## RECURSOS ADICIONALES

- **Documentación oficial:** [LangGraph Docs](https://langchain-ai.github.io/langgraph/)
- **Ejemplos GitHub:** [LangGraph Examples](https://github.com/langchain-ai/langgraph/tree/main/examples)
- **Tutorial interactivo:** [LangGraph Tutorials](https://langchain-ai.github.io/langgraph/tutorials/)